CER100 - Configure Cluster with Self Signed Certificates
========================================================

This notebook will:

1.  Generate a new Root CA in the Big Data Cluster
2.  Create new certificates for each endpoint (Management, Gateway,
    App-Proxy and Controller)
3.  Sign each new certificate with the new generated Root CA, except the
    Controller cert (which is signed with the existing cluster Root CA)
4.  Install each certificate into the Big Data Cluster
5.  Download the new generated Root CA into this machine’s Trusted Root
    Cerification Authorities certificate store.

All generated self-signed certificates will be stored in the controller
pod (at the `test_cert_store_root` location).

**NOTE: When CER010 runs (the 3rd notebook), look for the ‘Security
Warning’ dialog to pop up, and press ‘Yes’ to accept the installation of
the new Root CA into this machine’s certificate store.**

Upon completion of this notebook, all https:// access to the Big Data
Cluster from this machine (and any machine that installs the new Root
CA) will show as being secure.

The Notebook Runner chapter, will ensure CronJobs created (RUN003) to
run App-Deploy will install the cluster Root CA to allow securely
getting JWT tokens and the swagger.json.

Description
-----------

### Parameters

The parameters set here will override the default parameters set in each
individual notebook (`azdata notebook run` injects a `Parameters` cell
at runtime with the values passed in from the `-a` arguement)

In [ ]:
import getpass

common_name = "SQL Server Big Data Clusters Test CA"

country_name = "US"
state_or_province_name = "Illinois"
locality_name = "Chicago"
organization_name = "Contoso"
organizational_unit_name = "Finance"
email_address = f"{getpass.getuser()}@contoso.com"

days = "825" # the number of days to certify the certificates for

test_cert_store_root = "/var/opt/secrets/test-certificates"

### Define notebooks and their arguments

In [ ]:
import os
import copy

cer00_args = { "country_name": country_name, "state_or_province_name": state_or_province_name, "locality_name": locality_name, "organization_name": organization_name, "organizational_unit_name": organizational_unit_name, "common_name": common_name, "email_address": email_address, "days": days, "test_cert_store_root": test_cert_store_root }

cer02_args = copy.deepcopy(cer00_args)
cer02_args.pop("common_name") #  no common_name (as this is the service name set per endpoint)

cer04_args = { "test_cert_store_root": test_cert_store_root }

notebooks = [
    [ os.path.join("..", "common", "sop028-azdata-login.ipynb"), {} ],
    [ os.path.join("..", "cert-management", "cer001-create-root-ca.ipynb"), cer00_args ],
    [ os.path.join("..", "cert-management", "cer010-install-generated-root-ca-locally.ipynb"), cer04_args ],
    [ os.path.join("..", "cert-management", "cer020-create-management-service-proxy-cert.ipynb"), cer02_args ],
    [ os.path.join("..", "cert-management", "cer021-create-knox-cert.ipynb"), cer02_args ],
    [ os.path.join("..", "cert-management", "cer022-create-app-proxy-cert.ipynb"), cer02_args ],
    [ os.path.join("..", "cert-management", "cer023-create-controller-cert.ipynb"), cer02_args ],
    [ os.path.join("..", "cert-management", "cer030-sign-service-proxy-generated-cert.ipynb"), cer02_args ],
    [ os.path.join("..", "cert-management", "cer031-sign-knox-generated-cert.ipynb"), cer02_args ],
    [ os.path.join("..", "cert-management", "cer032-sign-app-proxy-generated-cert.ipynb"), cer02_args ],
    [ os.path.join("..", "cert-management", "cer033-sign-controller-generated-cert.ipynb"), cer02_args ],
    [ os.path.join("..", "cert-management", "cer040-install-service-proxy-cert.ipynb"), cer04_args ],
    [ os.path.join("..", "cert-management", "cer041-install-knox-cert.ipynb"), cer04_args ],
    [ os.path.join("..", "cert-management", "cer042-install-app-proxy-cert.ipynb"), cer04_args ],
    [ os.path.join("..", "cert-management", "cer043-install-controller-cert.ipynb"), cer04_args ],
    [ os.path.join("..", "cert-management", "cer050-wait-cluster-healthly.ipynb"), {} ]
]

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, hyperlinked suggestions, and scrolling updates on Windows
import sys
import os
import re
import json
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

retry_hints = {}
error_hints = {}
install_hint = {}

first_run = True
rules = None

def run(cmd, return_output=False, no_output=False, retry_count=0):
    """
    Run shell command, stream stdout, print stderr and optionally return output
    """
    MAX_RETRIES = 5
    output = ""
    retry = False

    global first_run
    global rules

    if first_run:
        first_run = False
        rules = load_rules()

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # Store this (i.e. kubectl, python etc.) to support binary context aware error_hints and retries
    #
    user_provided_exe_name = cmd_actual[0].lower()

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #    UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

    # To aid supportabilty, determine which binary file will actually be executed on the machine
    #
    which_binary = None

    # Special case for CURL on Windows.  The version of CURL in Windows System32 does not work to
    # get JWT tokens, it returns "(56) Failure when receiving data from the peer".  If another instance
    # of CURL exists on the machine use that one.  (Unfortunately the curl.exe in System32 is almost
    # always the first curl.exe in the path, and it can't be uninstalled from System32, so here we
    # look for the 2nd installation of CURL in the path)
    if platform.system() == "Windows" and cmd.startswith("curl "):
        path = os.getenv('PATH')
        for p in path.split(os.path.pathsep):
            p = os.path.join(p, "curl.exe")
            if os.path.exists(p) and os.access(p, os.X_OK):
                if p.lower().find("system32") == -1:
                    cmd_actual[0] = p
                    which_binary = p
                    break

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    if which_binary == None:
        which_binary = shutil.which(cmd_actual[0])

    if which_binary == None:
        if user_provided_exe_name in install_hint and install_hint[user_provided_exe_name] is not None:
            display(Markdown(f'HINT: Use [{install_hint[user_provided_exe_name][0]}]({install_hint[user_provided_exe_name][1]}) to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    print(f"START: {cmd} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #

    # Work around a infinite hang when a notebook generates a non-zero return code, break out, and do not wait
    #
    wait = True 

    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))

                                    wait = False
                                    break # otherwise infinite hang, have not worked out why yet.
                        else:
                            print(line, end='')
                            if rules is not None:
                                apply_expert_rules(line)

        if wait:
            p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'HINT: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    exit_code_workaround = 0 # WORKAROUND: azdata hangs on exception from notebook on p.wait()

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            line_decoded = line.decode()

            # azdata emits a single empty line to stderr when doing an hdfs cp, don't
            # print this empty "ERR:" as it confuses.
            #
            if line_decoded == "":
                continue
            
            print(f"STDERR: {line_decoded}", end='')

            if line_decoded.startswith("An exception has occurred") or line_decoded.startswith("ERROR: An error occurred while executing the following cell"):
                exit_code_workaround = 1

            if user_provided_exe_name in error_hints:
                for error_hint in error_hints[user_provided_exe_name]:
                    if line_decoded.find(error_hint[0]) != -1:
                        display(Markdown(f'HINT: Use [{error_hint[1]}]({error_hint[2]}) to resolve this issue.'))

            if rules is not None:
                apply_expert_rules(line_decoded)

            if user_provided_exe_name in retry_hints:
                for retry_hint in retry_hints[user_provided_exe_name]:
                    if line_decoded.find(retry_hint) != -1:
                        if retry_count < MAX_RETRIES:
                            print(f"RETRY: {retry_count} (due to: {retry_hint})")
                            retry_count = retry_count + 1
                            output = run(cmd, return_output=return_output, retry_count=retry_count)

                            if return_output:
                                return output
                            else:
                                return

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    # WORKAROUND: We avoid infinite hang above in the `azdata notebook run` failure case, by inferring success (from stdout output), so
    # don't wait here, if success known above
    #
    if wait: 
        if p.returncode != 0:
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')
    else:
        if exit_code_workaround !=0 :
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(exit_code_workaround)}.\n')


    print(f'\nSUCCESS: {elapsed}s elapsed.\n')

    if return_output:
        return output

def load_json(filename):
    with open(filename, encoding="utf8") as json_file:
        return json.load(json_file)

def load_rules():

    try:

        # Load this notebook as json to get access to the expert rules in the notebook metadata.
        #
        j = load_json("cer100-create-root-ca-install-certs.ipynb")

    except:
        pass # If the user has renamed the book, we can't load ourself.  NOTE: Is there a way in Jupyter, to know your own filename?

    else:
        if "metadata" in j and \
            "azdata" in j["metadata"] and \
            "expert" in j["metadata"]["azdata"] and \
            "rules" in j["metadata"]["azdata"]["expert"]:

            rules = j["metadata"]["azdata"]["expert"]["rules"]

            rules.sort() # Sort rules, so they run in priority order (the [0] element).  Lowest value first.

            # print (f"EXPERT: There are {len(rules)} rules to evaluate.")

            return rules

def apply_expert_rules(line):

    global rules

    for rule in rules:

        # rules that have 9 elements are the injected (output) rules (the ones we want).  Rules
        # with only 8 elements are the source (input) rules, which are not expanded (i.e. TSG029,
        # not ../repair/tsg029-nb-name.ipynb)
        if len(rule) == 9:
            notebook = rule[1]
            cell_type = rule[2]
            output_type = rule[3] # i.e. stream or error
            output_type_name = rule[4] # i.e. ename or name 
            output_type_value = rule[5] # i.e. SystemExit or stdout
            details_name = rule[6]  # i.e. evalue or text 
            expression = rule[7].replace("\\*", "*") # Something escaped *, and put a \ in front of it!

            # print(f"EXPERT: If rule '{expression}' satisfied', run '{notebook}'.")

            if re.match(expression, line, re.DOTALL):

                # print("EXPERT: MATCH: name = value: '{0}' = '{1}' matched expression '{2}', therefore HINT '{4}'".format(output_type_name, output_type_value, expression, notebook))

                match_found = True

                display(Markdown(f'HINT: Use [{notebook}]({notebook}) to resolve this issue.'))



print('Common functions defined successfully.')

# Hints for binary (transient fault) retry, (known) error and install guide
#
retry_hints = {'azdata': ['Endpoint sql-server-master does not exist', 'Endpoint livy does not exist', 'Failed to get state for cluster', 'Endpoint webhdfs does not exist', 'Adaptive Server is unavailable or does not exist', 'Error: Address already in use']}
error_hints = {'azdata': [['azdata login', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['The token is expired', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Reason: Unauthorized', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Max retries exceeded with url: /api/v1/bdc/endpoints', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Look at the controller logs for more details', 'TSG027 - Observe cluster deployment', '../diagnose/tsg027-observe-bdc-create.ipynb'], ['provided port is already allocated', 'TSG062 - Get tail of all previous container logs for pods in BDC namespace', '../log-files/tsg062-tail-bdc-previous-container-logs.ipynb'], ['Create cluster failed since the existing namespace', 'SOP061 - Delete a big data cluster', '../install/sop061-delete-bdc.ipynb'], ['Failed to complete kube config setup', 'TSG067 - Failed to complete kube config setup', '../repair/tsg067-failed-to-complete-kube-config-setup.ipynb'], ['Error processing command: "ApiError', 'TSG110 - Azdata returns ApiError', '../repair/tsg110-azdata-returns-apierror.ipynb'], ['Error processing command: "ControllerError', 'TSG036 - Controller logs', '../log-analyzers/tsg036-get-controller-logs.ipynb'], ['ERROR: 500', 'TSG046 - Knox gateway logs', '../log-analyzers/tsg046-get-knox-logs.ipynb'], ['Data source name not found and no default driver specified', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ["Can't open lib 'ODBC Driver 17 for SQL Server", 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Control plane upgrade failed. Failed to upgrade controller.', 'TSG108 - View the controller upgrade config map', '../diagnose/tsg108-controller-failed-to-upgrade.ipynb']]}
install_hint = {'azdata': ['SOP055 - Install azdata command line interface', '../install/sop055-install-azdata.ipynb']}

### Create a temporary directory to stage files

In [ ]:
# Create a temporary directory to hold configuration files

import tempfile

temp_dir = tempfile.mkdtemp()

print(f"Temporary directory created: {temp_dir}")

### Helper function for running notebooks with `azdata notebook run`

To pass ‘list’ types to `azdata notebook run --arguments`, flatten to
string

In [ ]:
# Define helper function 'run_notebook'

def run_notebook(name, arguments):
    for key, value in arguments.items():
        if isinstance(value, list):
           arguments[key] = str(value).replace("'", "")

    # --arguments have to be passed as \" \" quoted strings on Windows cmd line
    #
    # `app create` and `app run` can take a long time, so pass in a 30 minute cell timeout
    #
    arguments = str(arguments).replace("'", '\\"')  
    run(f'azdata notebook run -p "{os.path.join("..", "notebook-runner", name)}" --arguments "{arguments}" --output-path "{os.getcwd()}" --output-html --timeout 1800')

print("Function 'run_notebook' defined")

### Run the notebooks

In [ ]:
for notebook in notebooks:
    run_notebook(notebook[0], notebook[1])

print("Notebooks ran successfully.")

In [ ]:
print('Notebook execution complete.')

Related
-------

-   [CAN100 - Deploy all
    Canaries](../canary/can100-deploy-all-canaries.ipynb)

-   [CER001 - Generate a Root CA
    certificate](../cert-management/cer001-create-root-ca.ipynb)

-   [CER020 - Create Management Proxy
    certificate](../cert-management/cer020-create-management-service-proxy-cert.ipynb)

-   [CER021 - Create Knox
    certificate](../cert-management/cer021-create-knox-cert.ipynb)

-   [CER022 - Create App Proxy
    certificate](../cert-management/cer022-create-app-proxy-cert.ipynb)

-   [CER030 - Sign Management Proxy certificate with generated
    CA](../cert-management/cer030-sign-service-proxy-generated-cert.ipynb)

-   [CER031 - Sign Knox certificate with generated
    CA](../cert-management/cer031-sign-knox-generated-cert.ipynb)

-   [CER032 - Sign App-Proxy certificate with generated
    CA](../cert-management/cer032-sign-app-proxy-generated-cert.ipynb)

-   [CER040 - Install signed Management Proxy
    certificate](../cert-management/cer040-install-service-proxy-cert.ipynb)

-   [CER041 - Install signed Knox
    certificate](../cert-management/cer041-install-knox-cert.ipynb)

-   [CER042 - Install signed App-Proxy
    certificate](../cert-management/cer042-install-app-proxy-cert.ipynb)

-   [CER010 - Install generated Root CA
    locally](../cert-management/cer010-install-generated-root-ca-locally.ipynb)